In [2]:
import pandas as pd
import warnings

In [3]:
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_excel('../00_dataset/cohort analysis data.xlsx')
df.head()

,平台,店铺名称,客户昵称,付款时间,订单状态,支付金额,购买数量,省份
0,线上,小z杂货铺,入倩出入深,2019-09-01 00:10:04,交易成功,15.2,1,江苏省
1,线上,小z杂货铺,愛hya爱,2019-09-01 00:14:52,交易成功,8.4,1,广东省
2,线上,小z杂货铺,象95象大,2019-09-01 02:17:15,交易成功,8.4,1,辽宁省
3,线上,小z杂货铺,卡哇伊氛十,2019-09-01 03:37:28,交易成功,22.0,1,广西壮族自治区
4,线上,小z杂货铺,一只羊哈阿,2019-09-01 08:53:50,交易成功,85.0,1,辽宁省


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42713 entries, 0 to 42712
Data columns (total 8 columns):
平台      42713 non-null object
店铺名称    42713 non-null object
客户昵称    42713 non-null object
付款时间    40339 non-null datetime64[ns]
订单状态    42713 non-null object
支付金额    42713 non-null float64
购买数量    42713 non-null int64
省份      42713 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 2.6+ MB


In [10]:
df.loc[df['付款时间'].isnull(), :].head()

,平台,店铺名称,客户昵称,付款时间,订单状态,支付金额,购买数量,省份
40339,线上,小z杂货铺,爱购物nx,NaT,交易失败,97.8,1,浙江省
40340,线上,小z杂货铺,975ay,NaT,交易失败,117.3,2,浙江省
40341,线上,小z杂货铺,101呆阿,NaT,交易失败,144.5,2,新疆维吾尔自治区
40342,线上,小z杂货铺,489bt,NaT,交易失败,92.7,1,江苏省
40343,线上,小z杂货铺,姚琳儿姚姚,NaT,交易失败,8.4,1,广东省


In [15]:
print(df.loc[df['付款时间'].isnull(), :]['订单状态'].value_counts())
print(df.loc[df['付款时间'].isnull() == False, :]['订单状态'].value_counts())

交易失败    2374
Name: 订单状态, dtype: int64
交易成功    40339
Name: 订单状态, dtype: int64


In [6]:
# 清洗数据，去除缺失值，采用isnull()方法
order_data = df.loc[df['付款时间'].isnull() == False, :]

# 清洗数据，去除缺失值或采用dropna()方法
# order_data = df.dropna()
order_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40339 entries, 0 to 40338
Data columns (total 8 columns):
平台      40339 non-null object
店铺名称    40339 non-null object
客户昵称    40339 non-null object
付款时间    40339 non-null datetime64[ns]
订单状态    40339 non-null object
支付金额    40339 non-null float64
购买数量    40339 non-null int64
省份      40339 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 2.8+ MB


In [7]:
# 单月实现
order_data['时间标签'] = order_data['付款时间'].astype(str).str[:7]
order_data['时间标签'].value_counts().sort_index()

2019-09    2201
2019-10    8096
2019-11    6050
2019-12    6760
2020-01    7443
2020-02    9789
Name: 时间标签, dtype: int64

In [8]:
order_data.head()

,平台,店铺名称,客户昵称,付款时间,订单状态,支付金额,购买数量,省份,时间标签
0,线上,小z杂货铺,入倩出入深,2019-09-01 00:10:04,交易成功,15.2,1,江苏省,2019-09
1,线上,小z杂货铺,愛hya爱,2019-09-01 00:14:52,交易成功,8.4,1,广东省,2019-09
2,线上,小z杂货铺,象95象大,2019-09-01 02:17:15,交易成功,8.4,1,辽宁省,2019-09
3,线上,小z杂货铺,卡哇伊氛十,2019-09-01 03:37:28,交易成功,22.0,1,广西壮族自治区,2019-09
4,线上,小z杂货铺,一只羊哈阿,2019-09-01 08:53:50,交易成功,85.0,1,辽宁省,2019-09


In [18]:
# 10月用户情况
history_data = order_data.loc[order_data['时间标签'] == '2019-09', :]

month = '2019-10'
octb_order = order_data.loc[order_data['时间标签'] == month, :]
octb_customer = octb_order.groupby('客户昵称')['支付金额'].sum().reset_index()
print('10月订单量：', len(octb_order))
print('10月客户量：', len(octb_customer))
# octb_order.head()
# octb_customer.head()

# 10月新增客户量
newadd_customer = octb_customer.loc[octb_customer['客户昵称'].isin(history_data['客户昵称']) == False, :]
print('10月新增客户量：', len(newadd_customer))
newadd_customer.head()

10月订单量： 8096
10月客户量： 7336
10月新增客户量： 7083


,客户昵称,支付金额
0,0000栗,4.2
1,000ab,16.8
2,000al,16.8
3,000bb,16.8
4,000il,16.8


In [20]:
# 计算10月的留存情况
retention = []
for i in ['2019-11', '2019-12', '2020-01', '2020-02']:
    next_month = order_data.loc[order_data['时间标签'] == i, :]
    retention_num = octb_customer.loc[octb_customer['客户昵称'].isin(next_month['客户昵称']) == True, :]
    retention.append([i + '留存情况:', len(retention_num)])

retention

[['2019-11留存情况:', 628],
 ['2019-12留存情况:', 497],
 ['2020-01留存情况:', 490],
 ['2020-02留存情况:', 493]]

In [23]:
# 完整代码
#引入时间标签
month_lst = order_data['时间标签'].unique()
final = pd.DataFrame()

for i in range(len(month_lst) - 1):
    
    #构造和月份一样长的列表，方便后续格式统一
    count = [0] * len(month_lst)
    
    #筛选出当月订单，并按客户昵称分组
    target_month = order_data.loc[order_data['时间标签'] == month_lst[i],:]
    target_users = target_month.groupby('客户昵称')['支付金额'].sum().reset_index()
    
    #如果是第一个月份，则跳过（因为不需要和历史数据验证是否为新增客户）
    if i == 0:
        new_target_users = target_month.groupby('客户昵称')['支付金额'].sum().reset_index()
    else:
        #如果不是，找到之前的历史订单
        history = order_data.loc[order_data['时间标签'].isin(month_lst[:i]),:]
        #筛选出未在历史订单出现过的新增客户
        new_target_users = target_users.loc[target_users['客户昵称'].isin(history_data['客户昵称']) == False,:]
    
    #当月新增客户数放在第一个值中
    count[0] = len(new_target_users)
    
    #以月为单位，循环遍历，计算留存情况
    for j,ct in zip(range(i + 1,len(month_lst)),range(1,len(month_lst))):
        #下一个月的订单
        next_month = order_data.loc[order_data['时间标签'] == month_lst[j],:]
        next_users = next_month.groupby('客户昵称')['支付金额'].sum().reset_index()
        #计算在该月仍然留存的客户数量
        isin = new_target_users['客户昵称'].isin(next_users['客户昵称']).sum()
        count[ct] = isin
        
    #格式转置
    result = pd.DataFrame({month_lst[i]:count}).T
    
    #合并
    final = pd.concat([final,result])

final.columns = ['当月新增','+1月','+2月','+3月','+4月','+5月']

In [30]:
result = final.divide(final['当月新增'],axis = 0).iloc[:,1:]
result['当月新增'] = final['当月新增']
result

,+1月,+2月,+3月,+4月,+5月,当月新增
2019-09,0.123898,0.107248,0.081783,0.077865,0.080803,2042
2019-10,0.076098,0.060426,0.058450,0.060144,0.000000,7083
2019-11,0.109975,0.095211,0.080068,0.000000,0.000000,5283
2019-12,0.128350,0.099846,0.000000,0.000000,0.000000,5859
2020-01,0.132167,0.000000,0.000000,0.000000,0.000000,6401


In [26]:
final.to_excel('留存量分析表02.xlsx')
result.to_excel('留存率分析表02.xlsx')